# 1. ETL ➡ Extraer, Transformar y Cargar

Obtengo los datos a través del cliente interesado en que se le analicen los datos de su negocio. En este caso, es una farmacia comunitaria de Canarias, en Tenerife.
 
El cliente ha obtenido los datos a partir del software de su negocio, llamado [Unycop](https://www.unycop.es/). Los datos se encuentran en formato excel.

In [113]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

---

### Limpieza del primer archivo excel articulos

In [114]:
df = pd.read_excel('../data/data_raw/articulos.xlsx')
df = df.iloc[:, 1:]
df.columns = df.iloc[0]
df = df.iloc[1:, :]
df = df.reset_index(drop=True)


In [115]:
df.head()

,Código,Nombre,Familia,Mínimo,Existencias,P.V.P.,NaN,Ventas,Porcent.
0,702686,JARDIANCE 10 MG 30 COMP,Financiables,4,5,"49,53",NaN,"18.188,36","0,80%"
1,706414,TOUJEO 300 UI SOL INYECT 3 PLUMAS 1.5 M,Financiables,5,6,"48,68",NaN,"18.011,60","0,80%"
2,700588,FORXIGA 10 MG 28 COMP,Financiables,3,3,"44,25",NaN,"16.149,68","0,71%"
3,704632,TRULICITY 1.5 MG SOL INY 4 PLUMAS PREC,Financiables,2,2,"123,22",NaN,"15.895,38","0,70%"
4,935353,NEXIUM MUPS 40 MG 28 COMP,Financiables,5,4,"24,14",NaN,"14.966,80","0,66%"


In [116]:
df = df.drop_duplicates()

In [117]:
df = df.rename(columns={df.columns[6]: 'columna_vacia'})
columna_vacia = 'columna_vacia'
df = df.drop(columns=[columna_vacia], errors='ignore')


In [118]:
df.head()

,Código,Nombre,Familia,Mínimo,Existencias,P.V.P.,Ventas,Porcent.
0,702686,JARDIANCE 10 MG 30 COMP,Financiables,4,5,"49,53","18.188,36","0,80%"
1,706414,TOUJEO 300 UI SOL INYECT 3 PLUMAS 1.5 M,Financiables,5,6,"48,68","18.011,60","0,80%"
2,700588,FORXIGA 10 MG 28 COMP,Financiables,3,3,"44,25","16.149,68","0,71%"
3,704632,TRULICITY 1.5 MG SOL INY 4 PLUMAS PREC,Financiables,2,2,"123,22","15.895,38","0,70%"
4,935353,NEXIUM MUPS 40 MG 28 COMP,Financiables,5,4,"24,14","14.966,80","0,66%"


In [119]:
df = df.dropna()

In [120]:
df.head()

,Código,Nombre,Familia,Mínimo,Existencias,P.V.P.,Ventas,Porcent.
0,702686,JARDIANCE 10 MG 30 COMP,Financiables,4,5,"49,53","18.188,36","0,80%"
1,706414,TOUJEO 300 UI SOL INYECT 3 PLUMAS 1.5 M,Financiables,5,6,"48,68","18.011,60","0,80%"
2,700588,FORXIGA 10 MG 28 COMP,Financiables,3,3,"44,25","16.149,68","0,71%"
3,704632,TRULICITY 1.5 MG SOL INY 4 PLUMAS PREC,Financiables,2,2,"123,22","15.895,38","0,70%"
4,935353,NEXIUM MUPS 40 MG 28 COMP,Financiables,5,4,"24,14","14.966,80","0,66%"


In [121]:
columnas_a = ['Código', 'Nombre', 'Familia', 'Mínimo', 'Existencias', 'P.V.P.', 'Ventas', 'Porcent.']
eliminar = ['Código', 'Nombre', 'Familia', 'Mínimo', 'Existencias', 'P.V.P.', 'Ventas', 'Porcent.']

# Itera sobre las columnas y elimina las filas que contienen los valores especificados
for columna, valor in zip(columnas_a, eliminar):
    df = df[~(df[columna] == valor)]

In [122]:
df.to_excel('../data/data_clean/archivos.xlsx', index=False)

- Ánalisis del primer archivo

In [127]:
df.shape

(3352, 8)

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3352 entries, 0 to 3436
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Código       3352 non-null   object
 1   Nombre       3352 non-null   object
 2   Familia      3352 non-null   object
 3   Mínimo       3352 non-null   object
 4   Existencias  3352 non-null   object
 5   P.V.P.       3352 non-null   object
 6   Ventas       3352 non-null   object
 7   Porcent.     3352 non-null   object
dtypes: object(8)
memory usage: 235.7+ KB


In [126]:
df.describe().T

,count,unique,top,freq
0,,,,
Código,3352,3352,702686,1
Nombre,3352,3349,OMEPRAZOL STADA 20 MG 56 CAPS EFG,2
Familia,3352,9,Financiables,1652
Mínimo,3352,18,1,1371
Existencias,3352,89,1,1056
P.V.P.,3352,1551,"2,40",71
Ventas,3352,3052,"156,00",6
Porcent.,3352,48,"0,01%",1580


## Resumen